In [ ]:
from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
import extGoogleDrive as extGoogleDrive
# from extGoogleDrive import extGoogleDrive as GoogleDrive
from pprint import pprint
import pyAesCrypt
import io
import os
import hashlib
import numpy as np

from helpers import mkdir, driveList2localDict, ignore, strip_aes, file_size, size_to_upload
from helpers import obfuscate, deobfuscate

# gauth = GoogleAuth()
# gauth.LocalWebserverAuth() # client_secrets.json need to be in the same directory as the script
# drive = extGoogleDrive.extGoogleDrive(gauth)

## List local files

In [ ]:
def get_local_folder(root_path, verbose=False):
    folder_list= {root_path: ""}
    def localListFolder(path_structure, verbose=verbose):
        local_folders = [d for d in os.listdir(path_structure) if os.path.isdir(os.path.join(path_structure, d))]
        for f in local_folders:
            if ignore(f):
                if verbose: print(f"{path_structure}/{f}: {f['id']}")
                folder_list[f"{path_structure}/{f}"] = ""
                localListFolder(f"{path_structure}/{f}", verbose=verbose)
            
    localListFolder(root_path, verbose=verbose)
    return folder_list

get_local_folder("./mypydrive_1")

In [ ]:
def get_local_files(path):
    folders = get_local_folder(path)
    lfile_list = {}
    for fpath, _ in folders.items():
        lfile_list[fpath] = {d: "" for d in os.listdir(fpath) if ignore(d) and os.path.isfile(os.path.join(fpath, d))}
    return lfile_list
        
get_local_files("./mypydrive_1")

## List remote folders

In [ ]:
password = hashlib.sha256("foopassword".encode("utf-8")).hexdigest()
root_id = "1YJkqN-qtRHXnL4GvT3icGyl05_dcYTE6"

def get_remote_folders(verbose=True, decipher=True):
    folder_list= {"remote_root": "1YJkqN-qtRHXnL4GvT3icGyl05_dcYTE6"}
    if verbose: print(f"remote_root: '1YJkqN-qtRHXnL4GvT3icGyl05_dcYTE6'")
    
    def driveListFolder( parent, path_structure, verbose=verbose, decipher=decipher):
        remote_folders = drive.ListFile({'q': f"'{parent}' in parents and trashed=false and mimeType='application/vnd.google-apps.folder'"}).GetList()
        for f in remote_folders:
            title = f['title']
            if decipher: title = deobfuscate(password, title)
            if verbose: print(f"{path_structure}/{title}: {f['id']}")
            folder_list[f"{path_structure}/{title}"] = f['id']
    #         folder_list.append({f"{path_structure}/{title}": f['id']})
            driveListFolder(f['id'], f"{path_structure}/{title}", verbose, decipher)
        
    driveListFolder(root_id, "remote_root", decipher=decipher, verbose=verbose)
    return folder_list
            

# driveListFolder(root_id, "root", decipher=True, verbose=False)
r_folders = get_remote_folders(verbose=True)
r_folders

## List remote Files

In [ ]:
def get_remote_files(remote_folders):
    r_files = {}
    decipher = True
    for folder_name, folder_id in remote_folders.items():
        files = {}
        remote_files = drive.ListFile({'q': f"'{folder_id}' in parents and trashed=false and mimeType!='application/vnd.google-apps.folder'"}).GetList()
        for f in remote_files:
            title = f['title']
            if decipher: title = deobfuscate(password, strip_aes(title))
#             files.append({f"{title}": f['id']})
            files[f"{title}"] = f['id']
        r_files[folder_name] = files
    return r_files
        
r_files = get_remote_files(r_folders)
r_files

# To upload

## Drive class handler

In [ ]:
class DriveHandler:
    
    def __init__(self, local_root, password = "foopassword", bufferSize=64*1024, cipher=True):
        self.root_id = "1YJkqN-qtRHXnL4GvT3icGyl05_dcYTE6"
        self.local_root = local_root
        self.cache = None
        self.bufferSize = bufferSize
        self.password = hashlib.sha256(password.encode("utf-8")).hexdigest()
#         self.drive = drive
        self.cipher = cipher

d = DriveHandler("./mypydrive_1")

## Upload folders

In [ ]:
def folders_to_create(self, r_folders):
    L_folders = get_local_folder(self.local_root)
    to_create = []
    for folder in L_folders:
        rf = folder.replace(self.local_root, "remote_root")
        if rf not in r_folders.keys():
            to_create.append(rf)
    return to_create


folders_to_create(d, r_folders)

In [ ]:
def create_remote_folder(self, r_folders, cipher=True):
    to_create = folders_to_create(self, r_folders)
    for f in to_create:
        dirs = f.split("/")
        n = len(dirs)
        for i in range(n):
            remote_path = '/'.join(dirs[:i+1])
            if remote_path in r_folders.keys():
                continue
            else:
                parent_id = r_folders['/'.join(dirs[:i])]
                if cipher: child_name = obfuscate(self.password, dirs[i])
                child_folder = drive.CreateFile({'title': child_name, 
                                                  'parents':[{'id':parent_id}], 
                                                  'mimeType' : 'application/vnd.google-apps.folder'})
                child_folder.Upload()
                r_folders[remote_path] = child_folder['id']
                print(remote_path, child_name, parent_id, child_folder['id'])
            
create_remote_folder(d, r_folders)
r_folders

## Upload Files

In [ ]:
def encript_and_upload(self, fname, parent_id):
    fIn = open(fname, "rb")
    fCiph = io.BytesIO()
    pyAesCrypt.encryptStream(fIn, fCiph, self.password, self.bufferSize)

    if self.cipher: fname = obfuscate(self.password, fname)
    file = drive.CreateFile({'title': f"{fname}.aes", 
                                  'parents':[{'id': parent_id}]
                                 })
    file.SetContentBinary(fCiph)
    file.Upload() # Actual Upload

    fIn.close()
    return file['id']

In [ ]:
def files_to_upload(self):
    to_upload = []
    for folder, files in get_local_files(self.local_root).items():
        lf = folder
        rf = folder.replace(self.local_root, "remote_root")
        items = r_files.get(rf , {})
        for f in files:
            if f not in items.keys():
                to_upload.append( (f"{lf}/{f}", r_folders[rf]) )
#                 
    return to_upload
f2upload = files_to_upload(d)
f2upload

In [ ]:
def size_to_upload(f2upload, in_bytes=False):
    total = np.sum([file_size(f[0], in_bytes=True) for f in f2upload])
    if in_bytes:
        return total
    return file_size(total)

def file_size(fname, in_bytes=False, true_bytes=True):
    if type(fname) is str:
        byte = os.stat(fname).st_size
    else:
        byte = fname
    if in_bytes:
        return byte
    if byte == 0:
        return 0
    if true_bytes:
        power = np.log2(byte)
        if power < 10:
            return f"{byte}"
        elif power < 20:
            return f"{byte/2**10:.2f}Kb"
        elif power < 30:
            return f"{byte/2**20:.2f}Mb"
        else:
            return f"{byte/2**30:.2f}Gb"
    else:
        power = np.log10(byte)
        if power < 3:
            return f"{byte}"
        elif power < 6:
            return f"{byte/1e3:.2f}Kb"
        elif power < 9:
            return f"{byte/1e6:.2f}Mb"
        else:
            return f"{byte/1e9:.2f}Gb"

In [ ]:
def upload_files(self, f2upload):
    total = size_to_upload(f2upload, in_bytes=True)
    print(f'Total to upload: {file_size(total)}')
    for fname, parent_id in f2upload:
        file_id = encript_and_upload(self, fname, parent_id)
        total -= file_size(fname, in_bytes=True)
        print(f"{file_size(total):7} | {file_id} | {fname}")
    
upload_files(d, f2upload)

## Files to remove

In [ ]:
def folders_to_delete(self, r_folders):
    L_folders = get_local_folder(self.local_root)
    to_delete = []
    for folder, folder_id in r_folders.items():
        rf = folder.replace("remote_root", self.local_root)
        if rf not in L_folders.keys():
            to_delete.append((folder, folder_id))
    return to_delete

to_delete = folders_to_delete(d, r_folders)
to_delete

In [ ]:
def delete_remote_folders(self, to_delete):
    for folder, folder_id in to_delete:
        file = drive.CreateFile({'id': folder_id})
        file.Trash()

delete_remote_folders(d, to_delete)

In [ ]:
file = drive.CreateFile({'id': _id})
file.Trash()